# Langchain
### Quick start

In [3]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    base_url="http://localhost:12434/engines/v1",
    api_key="docker",
    temperature=0,
    model="ai/gemma3",
    stream_usage=True
)

# Build a basic agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    llm,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='20265bf4-4592-4e25-bd64-ff7a8708f26c'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 250, 'total_tokens': 293, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-n1FX31AZw7CRCpv1hVMvuUFq6YoxaHyq', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--a7d5c92b-d8e9-488a-ad70-21850d811fbd-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'SkbsEIBNzlv46GqBoiQ8dxWsAqyS9azN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 250, 'output_tokens': 43, 'total_tokens': 293, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content="It's always sunny in sf!", name='get_weather', id='570ebc85-9782-4

In [7]:
# Build a real-world agent

from dataclasses import dataclass

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.runtime import get_runtime

# Define system prompt
system_prompt = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

def get_user_location() -> str:
    """Retrieve user information based on user ID."""
    runtime = get_runtime(Context)
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Configure model
# model = init_chat_model(
#     "anthropic:claude-sonnet-4-5-20250929",
#     temperature=0
# )

# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
agent = create_agent(
    llm,
    # model=model,
    system_prompt=system_prompt,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


ResponseFormat(punny_response="Well, I'm not *cloudy* about it – Florida's weather is consistently bright! It's a real *heat*wave of a forecast.", weather_conditions=None)


In [9]:
response

{'messages': [HumanMessage(content='what is the weather outside?', additional_kwargs={}, response_metadata={}, id='415769cb-76c4-4b34-85d4-166f656a04a1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 590, 'total_tokens': 641, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ai/gemma3', 'system_fingerprint': 'b1-ca71fb9', 'id': 'chatcmpl-3RDAmHg7T65wPSM4ys882d9fCMZtYX8i', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--4b781211-650a-4ac0-9ce8-9721e684d6e3-0', tool_calls=[{'name': 'get_user_location', 'args': {}, 'id': 'qxICf2wyYosg7ZYHGASvSO1FnZNEHMip', 'type': 'tool_call'}], usage_metadata={'input_tokens': 590, 'output_tokens': 51, 'total_tokens': 641, 'input_token_details': {}, 'output_token_details': {}}),
  ToolMessage(content='Florida', name='get_user_location', id='74ebbfaf-9017-455d-b1fe-aead1